In [1]:
import math
import random
import json
from sklearn import linear_model
import sklearn
import csv
import gzip
import string
import copy
import scipy
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split

In [2]:
#Helpful functions
def readGz(path):
    for l in gzip.open(path, 'rb'):
        yield eval(l)

In [3]:
#Exploring the data
reviewSmall = [] #Users and a list of games they have reviewed
userToItem = [] #Users and a list of games they have downloaded
#User id's ^ directly correlated 

reviewBig = [] #Random assortment of reviews 
itemMetaData = [] #Data about each individual game
bundleData = [] #Data about each bundle
for d in readGz('../australian_user_reviews.json.gz'):
    reviewSmall.append(d)
    
for d in readGz('../australian_users_items.json.gz'):
    userToItem.append(d)
    
for d in readGz('../bundle_data.json.gz'):
    bundleData.append(d)
    
for d in readGz('../steam_games.json.gz'):
    itemMetaData.append(d)

# i = 0
# for d in readGz('../steam_reviews.json.gz'):
#     if i < 50000:
#         reviewBig.append(d)
#     else:
#         break
#     i+=1

In [336]:
relevantUserToItem = []
for u in userToItem:
    if len(u['items'])>=2:
        relevantUserToItem.append(u)

In [337]:
#Creating train and test sets
train=[]
test=[]
lengths=[]
item_lens=[]
for user in relevantUserToItem:
    items = user['items']
    trainIndex = round(0.7*len(items))
    item_lens.append(len(items))
    lengths.append(len(items)-round(0.7*len(items)))
    items_train = items[:trainIndex]
    items_test = items[trainIndex:]
    objTrain = {'user_id':user['user_id'], 'items': items_train}
    objTest = {'user_id':user['user_id'], 'items': items_test}
    train.append(objTrain)
    test.append(objTest)
    

## Baseline Model

In [345]:
itemHours = defaultdict(int)
for user in train:
    for item in user['items']:
        itemHours[item['item_id']]+=float(item['playtime_forever'])
    

In [350]:
mostPopular = [(itemHours[x], x) for x in itemHours]
mostPopular.sort()
mostPopular.reverse()
top20recs = [i[1] for i in mostPopular[:20]]

In [353]:
precisions=[]
for i, d in enumerate(test):
    currec = top20recs
    userid = d['user_id']
    userTestItems = ItemsForUserTest[userid]
    precisions.append(len(userTestItems.intersection(currec))/len(currec))

In [354]:
np.mean(np.array(precisions)>0)

0.0

## Predictions with Jaccard using the user reviews dataset

In [5]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [7]:
ItemsForUser=defaultdict(set)
for user in train:
    for item in user['items']:
        ItemsForUser[user['user_id']].add(item['item_id'])

In [8]:
UsersForItem = defaultdict(set)
for user in train:
    for item in user['items']:
        UsersForItem[item['item_id']].add(user['user_id'])

In [9]:
Top10ItemsForUser=defaultdict(list)
for d in train:
    user_id = d['user_id']
    items = d['items']
    items.sort(key=lambda x: x['playtime_forever'], reverse = True)
    itemids = [d['item_id'] for d in items]
    Top10ItemsForUser[user_id] = itemids[:10]

In [10]:
def mostSimilar(u, N):
    similarities = []
    items = set(Top10ItemsForUser[u])
    for u2 in ItemsForUser:
        if u2 == u: continue
        sim = Jaccard(items, set(Top10ItemsForUser[u2]))
        similarities.append((sim,u2))
    similarities.sort(reverse=True)
    return [u[1] for u in similarities[:N]]

In [103]:
top5SimilarUsers = defaultdict(list)
for d in train[:1000]:
    user_id = d['user_id']
    top5SimilarUsers[user_id] = mostSimilar(user_id, 5)

In [104]:
# df = pd.DataFrame(top5SimilarUsers)
# df.to_csv('MostSimilarUsersfirst1000.csv')

In [24]:
data = pd.read_csv('MostSimilarUsersfirst1000.csv')
data=data.T.drop('Unnamed: 0')
top5SimilarUsers = data.apply(lambda x: list([x[0], x[1], x[2], x[3], x[4]]),axis=1).to_dict() 

In [105]:
ItemsForUserTest=defaultdict(set)
for user in test:
    for item in user['items']:
        ItemsForUserTest[user['user_id']].add(item['item_id'])

In [106]:
top5SimilarUsersTest = defaultdict(list)
recommendations=[]
reclengths = []
for d in test[:1000]:
    userid = d['user_id']
    top5similar = top5SimilarUsers[userid]
    top50recs = Top10ItemsForUser[top5similar[0]]+Top10ItemsForUser[top5similar[1]]+Top10ItemsForUser[top5similar[2]]+Top10ItemsForUser[top5similar[3]]+Top10ItemsForUser[top5similar[4]]
    potentialrecs = set(top50recs)-ItemsForUser[userid]
    reclengths.append(len(potentialrecs))
    if len(potentialrecs)>20:
        potentialrecs = set(random.sample(potentialrecs, 20))
    recommendations.append(potentialrecs)

In [107]:
precisions=[]
for i, d in enumerate(test[:30]):
    currec = recommendations[i]
    userid = d['user_id']
    userTestItems = ItemsForUserTest[userid]
    precisions.append(len(userTestItems.intersection(currec))/len(currec))

In [108]:
np.mean(np.array(precisions)>0)

0.4666666666666667

## Predictions with Pearson using the Steam Reviews Dataset

In [223]:
reviewBig = []
for i, d in enumerate(readGz('../steam_reviews.json.gz')):
    reviewBig.append(d)
    if i==500000:
        break

In [224]:
trainPearson = reviewBig[:300000]
testPearson = []

In [225]:
for t in reviewBig[300000:]:
    if t['username'] in trainUsers:
        testPearson.append(t)

In [290]:
itemsPerUser2 = defaultdict(set)

for user in trainPearson:
    username, item = user['username'], user['product_id']
    itemsPerUser2[username].add(item)


In [292]:
trainRelevant = []
trainUsers = set()
itemsPerUserWithHours = defaultdict(list)
usersPerItemPearson = defaultdict(set)
itemsPerUserPearson = defaultdict(set)

for u in trainPearson:
    if len(itemsPerUser2[u['username']])>=2:
        try:
            hours = float(u['hours'])
            username, item = u['username'], u['product_id']
            trainRelevant.append(u)
            trainUsers.add(u['username'])
            usersPerItemPearson[item].add(username)
            itemsPerUserPearson[username].add(item)
            itemsPerUserWithHours[username].append({'item_id':item, 'hours':hours})
            rating=0       
            if hours>=50:
                rating=5
            elif hours>=20:
                rating=4
            elif hours>=8:
                rating=3
            elif hours>=2:
                rating=2
            elif hours>=0:
                rating=1
            ratingDict[(username,item)] = rating
        except:
            continue

        

In [293]:
testRelevant=[]
for t in testPearson:
    if t['username'] in trainUsers:
        testRelevant.append(t)

In [294]:
userAverages = {}
itemAverages = {}

for u in itemsPerUserPearson:
    rs = [ratingDict[(u,i)] for i in itemsPerUserPearson[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerItemPearson:
    rs = [ratingDict[(u,i)] for u in usersPerItemPearson[i]]
    itemAverages[i] = sum(rs) / len(rs)

In [307]:
def Pearson(u1, u2):
    # Between two users
    iBar1 = userAverages[u1]
    iBar2 = userAverages[u2]
    inter = itemsPerUserPearson[u1].intersection(itemsPerUserPearson[u2])
    numer = 0
    denom1 = 0
    denom2 = 0
    for i in inter:
        numer += (ratingDict[(u1,i)] - iBar1)*(ratingDict[(u2,i)] - iBar2)
    for i1 in itemsPerUserPearson[u1]:
        denom1 += (ratingDict[(u1,i1)] - iBar1)**2
    for i2 in itemsPerUserPearson[u2]:
        denom2 += (ratingDict[(u2,i2)] - iBar2)**2
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0: return 0
    return numer / denom

In [308]:
def mostSimilarPearson(u, N):
    similarities = []
    items = itemsPerUserPearson[u]
    for u2 in list(itemsPerUserPearson.keys()):
        if u2 == u: continue
        sim = Pearson(u, u2)
        #sim = Pearson(i, i2) # Could use alternate similarity metrics straightforwardly
        similarities.append((sim,u2))
    similarities.sort(reverse=True)
#     return similarities[:N]
    return [u[1] for u in similarities[:N]]

In [329]:
top100test = testRelevant[:100]

In [330]:
top10SimilarUsersPearson = defaultdict(list)
for user in top100test:
    user_id = user['username']
    top10SimilarUsersPearson[user_id] = mostSimilarPearson(user_id, 10)

In [324]:
ItemsForUserTestPearson=defaultdict(set)
for user in testRelevant:
    ItemsForUserTestPearson[user['username']].add(user['product_id'])

In [325]:
Top10ItemsForUser=defaultdict(list)
for user_id in list(itemsPerUserWithHours.keys()):
    items = itemsPerUserWithHours[user_id]
    items.sort(key=lambda x: x['hours'], reverse = True)
    itemids = [d['item_id'] for d in items]
    Top10ItemsForUser[user_id] = itemids[:10]

In [331]:
recommendations=[]
reclengths = []
for user in top100test:
    user_id = user['username']
    topsimilar = top10SimilarUsersPearson[user_id]
    toprecs = []
    for user in topsimilar:
        toprecs=toprecs+Top10ItemsForUser[user]
        
    potentialrecs = set(toprecs)-itemsPerUserPearson[user_id]
    reclengths.append(len(potentialrecs))
    if len(potentialrecs)>20:
        potentialrecs = set(random.sample(potentialrecs, 20))
    recobj = {'user_id':user_id, 'recommendations':potentialrecs}
    recommendations.append(recobj)

In [332]:
precisionsPearson=[]
for r in recommendations:
    items=ItemsForUserTestPearson[r['user_id']]
    if len(r['recommendations'])==0:
        precisionsPearson.append(0)
    else:
        precisionsPearson.append(len(set(items).intersection(r['recommendations']))/len(r['recommendations']))

In [334]:
np.mean(np.array(precisionsPearson)>0)

0.15